In [2]:
import tensorflow as tf
import numpy as np
import os
import os.path
import transfer_learning  # This module contains large amounts of pre-written code

In [3]:
# Ensure target log dir exists
INCEPTION_LOG_DIR = './tmp/inception_v3_log'
if not os.path.exists(INCEPTION_LOG_DIR):
    os.makedirs(INCEPTION_LOG_DIR)

In [4]:
# For performance reasons, we only use 150 images per class.
training_images, testing_images, label_maps = transfer_learning.create_image_lists(
    './data/adni_photos', testing_percentage=10, max_number_images=150)

INFO:tensorflow:Looking for images in 'AD'
INFO:tensorflow:Looking for images in 'CN'
INFO:tensorflow:Looking for images in 'EMCI'
INFO:tensorflow:Looking for images in 'LMCI'


In [77]:
# Create the inception model.
graph, bottleneck, resized_input, softmax = transfer_learning.create_model()

In [78]:
# Use a summary writer to write the loaded graph and display it in tensorboard.
with graph.as_default():
    jpeg_data, decoded_image = transfer_learning.make_jpeg_decoding()
    summary_writer = tf.summary.FileWriter(os.path.join(INCEPTION_LOG_DIR, 'original'), graph)
    summary_writer.close()
    

In [79]:
with graph.as_default():
    # Define summaries for tensorboard

    output_summary = tf.summary.histogram('softmax_output', softmax)  # histogram of softmax output
    input_summary = tf.summary.image('input_image', resized_input)  # display image of input
    bottleneck_summary = tf.summary.histogram('bottleneck_activations', bottleneck)  # second to last layer output
    #-- 
    summary_op = tf.summary.merge_all()

In [80]:
def classify_image(session, image_path, summary_op):
    
    # Open single image file and extract data
    with open(image_path, 'rb') as f:
        image_data = f.read()
    
 
    # Create Summary writer object
    summary_writer = tf.summary.FileWriter(os.path.join(INCEPTION_LOG_DIR, 'original'), graph)
        
    # Run pre-defined pipeline to decode & resize raw image data into decoded_image 
    image = session.run(decoded_image, feed_dict={jpeg_data: image_data})
        
    # Run input through network and obtain output from last layer, defined by create_model() call above.

    softmax_output, summary = session.run([softmax, summary_op], feed_dict={resized_input: image})
    
    # Log summary & terminate writer in the usual way.
    summary_writer.add_summary(summary)
    summary_writer.close()
    #--
    
    # Return label
    return(np.argmax(softmax_output),softmax_output)

In [8]:
image_path = os.path.join(os.getcwd(), 'data', 'adni_photos', 'AD', 'ADNI_130_S_4984_MR_MT1__N3m_Br_20121210160320615_S172131_I350508.nii-slice128.jpg')
with graph.as_default():
    with tf.Session() as session:
        # We classify the image and print the label
        imagenet_label,_ = classify_image(session, image_path, summary_op)
print(imagenet_label)

800


In [33]:
image_path = os.path.join(os.getcwd(), 'data', 'adni_photos', 'AD', 'ADNI_130_S_4984_MR_MT1__N3m_Br_20121210160320615_S172131_I350508.nii-slice128.jpg')
with graph.as_default():
    with tf.Session() as session:
        imagenet_label,softmax_output = classify_image(session, image_path, summary_op)
print(imagenet_label)

800


In [9]:
def compute_bottleneck(session, image_data):

    image = session.run(decoded_image, feed_dict={jpeg_data: image_data})
    return session.run(bottleneck, feed_dict={resized_input: image})
    #--

In [10]:
# This cell generates all the bottlenecks. Warning: it may take a while
# The results are cached so you only need to do it once -- if you change
# your compute_bottleneck function, you will need to delete the existing
# files to force the notebook to regenerate the bottlenecks (they are
# found in ./data/bottlenecks)

with graph.as_default():
    with tf.Session() as session:
        transfer_learning.cache_bottlenecks(compute_bottleneck, session, training_images)

Saved 1/533 bottlenecks
Saved 21/533 bottlenecks
Saved 41/533 bottlenecks
Saved 61/533 bottlenecks
Saved 81/533 bottlenecks
Saved 101/533 bottlenecks
Saved 121/533 bottlenecks
Saved 141/533 bottlenecks
Saved 161/533 bottlenecks
Saved 181/533 bottlenecks
Saved 201/533 bottlenecks
Saved 221/533 bottlenecks
Saved 241/533 bottlenecks
Saved 261/533 bottlenecks
Saved 281/533 bottlenecks
Saved 301/533 bottlenecks
Saved 321/533 bottlenecks
Saved 341/533 bottlenecks
Saved 361/533 bottlenecks
Saved 381/533 bottlenecks
Saved 401/533 bottlenecks
Saved 421/533 bottlenecks
Saved 441/533 bottlenecks
Saved 461/533 bottlenecks
Saved 481/533 bottlenecks
Saved 501/533 bottlenecks
Saved 521/533 bottlenecks
Done computing bottlenecks!


In [68]:
# This loads the training data as a matrix of training examples
# and a vector of labels
training_data_set = transfer_learning.create_training_dataset(training_images)

### Training the classifier

In [81]:
def make_final_layers(bottleneck_tensor, num_classes):
   
    bottleneck_tensor_size = int(bottleneck.shape[1])
    
    with tf.variable_scope('input'):
        # This is the input for the bottleneck. It is created
        # as a placeholder with default. During training, we will
        # be passing in the bottlenecks, but during evaluation,
        # the value will be propagated from the bottleneck computed
        # from the image using the full network.
        bottleneck_input = tf.placeholder_with_default(
            bottleneck_tensor,
            [None, bottleneck_tensor_size],
            'bottleneck_input')
        
        # This is the input for the label (integer, 1 to number of classes)
        label_input = tf.placeholder(tf.int64, [None], name='label_input')
    
    
    # -- Start student must write
    # Define weights, biases, and logit transforms
    logits = tf.layers.dense(bottleneck_input, num_classes)
    # Compute the cross entropy loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=label_input, logits=logits)
    # Create a summary for the loss
    loss_summary = tf.summary.scalar('cross_entropy', loss)
    # Create a Gradient Descent Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.1)
    # Obtain a function which performs a single training step
    train_step = optimizer.minimize(loss)
    # -- End student must write
    
    return bottleneck_input, label_input, logits, train_step, loss_summary

In [82]:
def compute_accuracy(labels, logits):
    
    
    # Collapse logit output to scalar labels
    predicted_label = tf.argmax(logits, 1)
    # Compute accuracy
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted_label, labels), tf.float32))
    # create summary of accuracy
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
    #--
    
    return accuracy, accuracy_summary

In [83]:
# We create the necessary operations to fine tune the model.

with graph.as_default():
    bottleneck_input, label_input, logits, train_step, loss_summary = make_final_layers(bottleneck, len(label_maps))
    accuracy, accuracy_summary = compute_accuracy(label_input, logits)
    summary_op = tf.summary.merge([loss_summary, accuracy_summary])

In [6]:
def execute_train_step(session: tf.Session, summary_writer: tf.summary.FileWriter, current_step: int):

    _, ac, summary = session.run((train_step, accuracy, summary_op),
                       feed_dict={bottleneck_input: training_data_set['bottlenecks'],
                                  label_input: training_data_set['labels']
                                 })
    
    summary_writer.add_summary(summary, current_step)
    
    if current_step % 10 == 0:
        print('Accuracy at step {0} is {1}'.format(current_step, ac))
        

In [5]:
def evaluate_images(session: tf.Session, images_jpeg_data: [bytes], labels: [int]):
  
    correct = []
    
    for label, jpeg in zip(labels, images_jpeg_data):
        image_data = session.run(decoded_image, feed_dict={jpeg_data: jpeg})
        ac = session.run(accuracy, feed_dict={resized_input: image_data, label_input: [label]})
        correct.append(ac)
    
    return np.mean(correct)

In [86]:
with graph.as_default():
    with tf.Session() as session:
        print('------------- Starting training ----------------')
        session.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter(os.path.join(INCEPTION_LOG_DIR, 'retrained'), graph)
        for i in range(200):
            execute_train_step(session, summary_writer, i)
        summary_writer.close()  
        print('------------- Training done! -------------------')
        print('---------- Loading testing data ----------------')
        tlabels, timages = transfer_learning.get_testing_data(testing_images)
        print('----------- Evaluating on testing --------------')
        
        eval_accuracy = evaluate_images(session, timages, tlabels)
        print('Evaluation accuracy was: {0}'.format(eval_accuracy))

------------- Starting training ----------------
Accuracy at step 0 is 0.5910780429840088
Accuracy at step 10 is 0.48327139019966125
Accuracy at step 20 is 0.5315985083580017
Accuracy at step 30 is 0.643122673034668
Accuracy at step 40 is 0.5464683771133423
Accuracy at step 50 is 0.6579925417900085
Accuracy at step 60 is 0.6356877088546753
Accuracy at step 70 is 0.5687732100486755
Accuracy at step 80 is 0.643122673034668
Accuracy at step 90 is 0.654275119304657
Accuracy at step 100 is 0.5836431384086609
Accuracy at step 110 is 0.6617100238800049
Accuracy at step 120 is 0.6579925417900085
Accuracy at step 130 is 0.535315990447998
Accuracy at step 140 is 0.7063196897506714
Accuracy at step 150 is 0.5910780429840088
Accuracy at step 160 is 0.6914498209953308
Accuracy at step 170 is 0.6691449880599976
Accuracy at step 180 is 0.5576208233833313
Accuracy at step 190 is 0.7137546539306641
------------- Training done! -------------------
---------- Loading testing data ----------------
-------